In [1]:
import jieba
import pandas as pd
import re

In [2]:
df = pd.read_csv('./icook_sort.csv')

In [3]:
df.head()

,_id,RecipeID,CategoryName,SubCategoryID,SubCategoryName,RecipeName,Description,AuthorID,Author,Servings,CookingTime,Ingredient,CookingSteps,LikeStat,Datetime,Preview,URL,Ingredient_Pro
0,60646cc365cd68e35c01e814,170303,家常菜色,"['104', '602']","['烤肉', '中秋料理']",綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒，其中照燒烤肉醬只要幾種材料組合，不必擔心市售烤肉醬有過多添...,100002279834418,胖仙女,4人份,45分鐘,"{'去骨雞腿': '4片', '起士明太子醬': '適量', '蔥白': '適量', '檸檬...",['1. 雞腿肉切小塊的同時，切去皮及多餘的脂肪，取其中2/3的雞肉，加入所有的醃料拌勻，醃...,662,2016-09-11,2.4 萬 瀏覽,https://icook.tw/recipes/170303,"{'去骨雞腿': ['4', '片'], '起士明太子醬': ['1', '適量'], '蔥..."
1,60646cc365cd68e35c01e815,169048,家常菜色,"['104', '77']","['烤肉', '宵夜']",燒烤味增秋葵透抽捲,秋葵正當季，適逢中秋節（烤肉節）腳步漸漸接近，來道『烤味增秋葵透抽』吧！\n簡單備料，有彈性...,charminglydia,懶廚房,2人份,20分鐘,"{'透抽': '1尾', '黃秋葵': '6個', '味增燒烤醬': '適量', '白芝麻'...","['1. 食材大集合。', '2. 黃秋葵先在鹽上滾一滾，如此讓表面絨毛軟化，較好入口。',...",675,2016-09-01,1.7 萬 瀏覽,https://icook.tw/recipes/169048,"{'透抽': ['1', '尾'], '黃秋葵': ['6', '個'], '味增燒烤醬':..."
2,60646cc365cd68e35c01e816,155271,家常菜色,"['104', '77', '602']","['烤肉', '宵夜', '中秋料理']",沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串，用沙嗲醬醃雞肉，然後放入烤箱烤，這可令雞肉更嫰滑多汁。再加上...,MrsPsKitchen,Mrs P's Kitchen,4人份,45分鐘,"{'雞肉': '600克', '沙嗲醬': '1大匙', '醬油': '1小匙', '糖':...","['1. 將雞肉切小塊，然後用醃料醃雞肉最少一個小時', '2. 將竹籤浸在熱水5分鐘，這可...","1,669",2016-06-02,3.3 萬 瀏覽,https://icook.tw/recipes/155271,"{'雞肉': ['600', '克'], '沙嗲醬': ['1', '大匙'], '醬油':..."
3,60646cc365cd68e35c01e817,90086,家常菜色,"['104', '602']","['烤肉', '中秋料理']",甜柿烤肉串,霜降吃柿子！甜柿營養豐富，卻不適合空腹吃。與肉類搭配，解油膩又清爽。簡單做法，最適合週末小派對！,jj5colorkitchen,jj5色廚,3人份,30分鐘,"{'火鍋白肉片': '6-7 片', '甜柿': '1 顆', '百里香': '適量', '...",['1. 這種豬肉片就是平常酸菜白肉鍋用的白肉片，等於沒薰過的培根肉，形狀規則，油脂豐富，比...,671,2014-10-25,1.8 萬 瀏覽,https://icook.tw/recipes/90086,"{'火鍋白肉片': ['6-7', '片'], '甜柿': ['1', '顆'], '百里香..."
4,60646cc365cd68e35c01e818,337797,家常菜色,"['104', '61']","['烤肉', '韓式']",韓式烤肉飯,百吃不膩的韓式烤肉飯，保證讓人胃口大開！無論配麵、配飯都很搭。 作法超級簡單！適合料理初學者...,1f2cb7b4bee5e7d2,幸福餐桌,2人份,45分鐘,"{'沙朗牛肉片': '400克', '蒜': '3瓣', '洋蔥': '1顆', '蘋果':...","['1. 以刀背將牛肉片敲打鬆軟', '2. 牛肉片切成小塊狀', '3. 將蒜、奇異果、蘋...",53,2020-05-23,"8,026 瀏覽",https://icook.tw/recipes/337797,"{'沙朗牛肉片': ['400', '克'], '蒜': ['3', '瓣'], '洋蔥':..."


In [4]:
df_TFIDF = df[['RecipeID','RecipeName', 'Description']]

In [5]:
# 將空值資料刪除
df_TFIDF=df_TFIDF.dropna()

In [6]:
# 將非空值的塞選出來做分析
df_TFIDF.head()

,RecipeID,RecipeName,Description
0,170303,綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒，其中照燒烤肉醬只要幾種材料組合，不必擔心市售烤肉醬有過多添...
1,169048,燒烤味增秋葵透抽捲,秋葵正當季，適逢中秋節（烤肉節）腳步漸漸接近，來道『烤味增秋葵透抽』吧！\n簡單備料，有彈性...
2,155271,沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串，用沙嗲醬醃雞肉，然後放入烤箱烤，這可令雞肉更嫰滑多汁。再加上...
3,90086,甜柿烤肉串,霜降吃柿子！甜柿營養豐富，卻不適合空腹吃。與肉類搭配，解油膩又清爽。簡單做法，最適合週末小派對！
4,337797,韓式烤肉飯,百吃不膩的韓式烤肉飯，保證讓人胃口大開！無論配麵、配飯都很搭。 作法超級簡單！適合料理初學者...


In [7]:
for n, Description in enumerate(df_TFIDF['Description']):
    Description = re.sub(r'[\W+]|[a-zA-Z0-9]', '', Description) # 特殊符號、英文、數字 刪除
    df_TFIDF.loc[n, 'Description'] = Description
df_TFIDF.head()

,RecipeID,RecipeName,Description
0,170303.0,綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒其中照燒烤肉醬只要幾種材料組合不必擔心市售烤肉醬有過多添加物...
1,169048.0,燒烤味增秋葵透抽捲,秋葵正當季適逢中秋節烤肉節腳步漸漸接近來道烤味增秋葵透抽吧簡單備料有彈性的透抽配上脆脆的秋葵...
2,155271.0,沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串用沙嗲醬醃雞肉然後放入烤箱烤這可令雞肉更嫰滑多汁再加上自己調的...
3,90086.0,甜柿烤肉串,霜降吃柿子甜柿營養豐富卻不適合空腹吃與肉類搭配解油膩又清爽簡單做法最適合週末小派對
4,337797.0,韓式烤肉飯,百吃不膩的韓式烤肉飯保證讓人胃口大開無論配麵配飯都很搭作法超級簡單適合料理初學者或者想快速上...


In [8]:
# 匯入自定義字典
jieba.set_dictionary('jieba_data/dict.txt')

In [9]:
# 匯入停用字典
with open('./jieba_data/chinese_sw.txt', 'r', encoding='utf-8') as f:
    stop_words_list = f.read().split('\n')
stop_words_list[:10]

['啊', '阿', '哎', '哎呀', '哎喲', '唉', '俺', '有點', '一直', '俺們']

In [10]:
Description_TFIDF = []
for Description in df_TFIDF['Description']:
    Description_Cut = jieba.lcut(Description)
    Filter_Description = []
    
    # 將 Description內的每個斷詞用停用字典來過濾
    for Each_word in Description_Cut:
        if Each_word not in stop_words_list:
            Filter_Description.append(Each_word)
    Filter_Description = ' '.join(Filter_Description) # 將過濾好的 Filter_Description每個詞用空白隔開
#     print(Filter_Description)
    Description_TFIDF.append(Filter_Description) # 將每篇食譜的 Description 放在一個 List 準備做 tfidf
#     print(Filter_Description)
#     print('-----------')

Building prefix dict from C:\Users\CEB10201\PycharmProjects\Python_Recipe_ETL\jieba_data\dict.txt ...
Loading model from cache C:\Users\CEB10201\AppData\Local\Temp\jieba.u8d2bce68ea74a61e4827369d82e5dd51.cache
Loading model cost 0.486 seconds.
Prefix dict has been built successfully.


In [11]:
#test
Description_TFIDF[:5]

['平底鍋 烤箱 完成 雞肉 串燒 照燒 烤肉醬 幾種 材料 組合 擔心 市售 烤肉醬 過多 添加物 作法 輕輕鬆鬆 做出 居酒屋 料理 喔',
 '秋葵 正當 季 適逢 中秋節 烤肉 節 腳步 漸漸 接近 來道 烤味增 秋葵 透抽 備料 彈性 透抽 配上 脆脆 秋葵 清爽 味增 燒烤 醬 腸胃 健康',
 '充滿 東南亞 風味 沙嗲 烤雞 串用 沙嗲 醬 醃 雞肉 放入 烤箱 烤 這可令 雞肉 嫰 滑 多汁 再 調 沙嗲 沾醬 超惹味 一串 接一串 下午茶 點或 派對 小吃 選擇 大受 歡迎 到 專頁 更多 食譜 英國 生活 分享',
 '霜降 吃 柿子 甜柿 營養 豐富 卻 適合 空腹 吃 肉類 搭配 解 油膩 清爽 做法 適合 週末 小 派對',
 '百吃 不膩 韓式烤肉 飯 保證 人 胃口 大開 配 麵 配飯 搭 作法 超級 適合 料理 學者 想 快速 上菜 吃 貨們']

In [12]:
len(Description_TFIDF)

26179

In [13]:
# 使用 sklearn內的 TfidfVectorizer模型找出每個詞在每篇食譜 Description內的 tdidf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_model= vectorizer.fit_transform(Description_TFIDF)

In [14]:
tfidf_model # 26179筆資料x35697不重覆的字的tfidf

<26179x35693 sparse matrix of type '<class 'numpy.float64'>'
	with 405200 stored elements in Compressed Sparse Row format>

In [15]:
# 矩陣內的每個 list代表每篇食譜所出現的每字的 tfidf
tfidf_model.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
# test 找出用模型跑出的所有食譜會出現的特徵(出現不同的字)
vectorizer.get_feature_names()[:5]

['__', '___', '____', '_____', '_______']

In [17]:
# 再次清洗將模型跑出的部分特徵加入stop wrod
#使用正規表達式將想要加入的特徵篩選出來

#'一'開頭的特徵
# stopwordadd = re.compile(r'^一.*')
# for x in vectorizer.get_feature_names():
#     if stopwordadd.findall(x) != []:
#         with open('./jieba_data/stop_words_add.txt', 'a', encoding='utf-8') as f:
#             f.write('\n')
#             f.write(x)            

In [18]:
# 將每篇食譜出現的字的 tfidf按權重高→低排序，並取前三個
TotalRecipeTFIDF = []
for RecipeTFIDF in tfidf_model.toarray():
    tmplist = []
    
    # 將 model 內的array 做轉換 ex. [[(word1,tdidf1),(word2,tdidf2)...],[(word1,tdidf1),(word2,tdidf2)...]...]
    for n, word in enumerate(vectorizer.get_feature_names()):
        WordTFIDF = (word, RecipeTFIDF[n])
        tmplist.append(WordTFIDF)
   
    tmplist = sorted(tmplist, key=lambda tmplist : tmplist[1], reverse=True)[:3]  # 轉換後 list 內每個 touple 取 tfidf 前三高的值
    TotalRecipeTFIDF.append(tmplist)
    
TotalRecipeTFIDF[:10]

[[('烤肉醬', 0.5290745826238659),
  ('串燒', 0.2815267993439184),
  ('幾種', 0.2632328527909649)],
 [('秋葵', 0.5168516480350709),
  ('透抽', 0.4051248538412719),
  ('烤味增', 0.2579841376057032)],
 [('沙嗲', 0.5932817630288907),
  ('這可令', 0.23364304909948497),
  ('接一串', 0.2246031936395523)],
 [('空腹', 0.4084285135927628),
  ('甜柿', 0.39262605448762755),
  ('霜降', 0.3596037560928037)],
 [('貨們', 0.4097896250009421),
  ('韓式烤肉', 0.3608151591730732),
  ('百吃', 0.32693459373808964)],
 [('好吃', 1.0), ('__', 0.0), ('___', 0.0)],
 [('雷同', 0.48199400026098876),
  ('某家', 0.45443229318951356),
  ('居酒屋', 0.3765933343336249)],
 [('沙爹', 0.7199047269005894),
  ('印尼', 0.37225008823527167),
  ('峇里島', 0.22409614323864818)],
 [('茭白筍', 0.6812901972183018),
  ('烤分', 0.37948975990015754),
  ('烤過', 0.3121367217713536)],
 [('升火', 0.42383415075125974),
  ('滿身', 0.3451343080313103),
  ('不失', 0.29615331372394554)]]

In [19]:
df_TFIDF['WordTFIDF'] = 0
df_TFIDF.head()    

,RecipeID,RecipeName,Description,WordTFIDF
0,170303.0,綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒其中照燒烤肉醬只要幾種材料組合不必擔心市售烤肉醬有過多添加物...,0
1,169048.0,燒烤味增秋葵透抽捲,秋葵正當季適逢中秋節烤肉節腳步漸漸接近來道烤味增秋葵透抽吧簡單備料有彈性的透抽配上脆脆的秋葵...,0
2,155271.0,沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串用沙嗲醬醃雞肉然後放入烤箱烤這可令雞肉更嫰滑多汁再加上自己調的...,0
3,90086.0,甜柿烤肉串,霜降吃柿子甜柿營養豐富卻不適合空腹吃與肉類搭配解油膩又清爽簡單做法最適合週末小派對,0
4,337797.0,韓式烤肉飯,百吃不膩的韓式烤肉飯保證讓人胃口大開無論配麵配飯都很搭作法超級簡單適合料理初學者或者想快速上...,0


In [20]:
for n,RecipeTFIDF in enumerate(TotalRecipeTFIDF):
    tmplist = []
    for x in RecipeTFIDF:
        tmplist.append(x[0])
#     print(tmplist)
#     print('--------------')
    df_TFIDF.loc[n,'WordTFIDF'] = str(tmplist)
    
df_TFIDF.head()

,RecipeID,RecipeName,Description,WordTFIDF
0,170303.0,綜合雞肉串燒,用平底鍋或烤箱就能完成的雞肉串燒其中照燒烤肉醬只要幾種材料組合不必擔心市售烤肉醬有過多添加物...,"['烤肉醬', '串燒', '幾種']"
1,169048.0,燒烤味增秋葵透抽捲,秋葵正當季適逢中秋節烤肉節腳步漸漸接近來道烤味增秋葵透抽吧簡單備料有彈性的透抽配上脆脆的秋葵...,"['秋葵', '透抽', '烤味增']"
2,155271.0,沙嗲烤雞肉串,這個充滿東南亞風味的沙嗲烤雞串用沙嗲醬醃雞肉然後放入烤箱烤這可令雞肉更嫰滑多汁再加上自己調的...,"['沙嗲', '這可令', '接一串']"
3,90086.0,甜柿烤肉串,霜降吃柿子甜柿營養豐富卻不適合空腹吃與肉類搭配解油膩又清爽簡單做法最適合週末小派對,"['空腹', '甜柿', '霜降']"
4,337797.0,韓式烤肉飯,百吃不膩的韓式烤肉飯保證讓人胃口大開無論配麵配飯都很搭作法超級簡單適合料理初學者或者想快速上...,"['貨們', '韓式烤肉', '百吃']"


In [22]:
df_TFIDF=df_TFIDF.dropna()

In [23]:
df_TFIDF.to_csv('icook_Description_TF-IDF.csv', encoding='utf-8-sig')